# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [ ]:
import joblib

from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core import Dataset

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
ws = Workspace.from_config()
print(ws)

# dataset key info
key = 'nba_3pt_data'
data_desc = 'NBA game data from 2015 to 2023 containing 3 point percentages.'

# checking if dataset is already registered
if key in ws.datasets.keys():
    nba_dataset = ws.datasets(key)
else:
    # get data from url
    nba_data_url = 'https://raw.githubusercontent.com/mattwatson50/udacity_azure_ml_final/main/data/nba_3pt_data.csv'
    dataset = Dataset.Tabular.from_delimited_files(nba_data_url)        
    # register dataset
    dataset = dataset.register(workspace=ws,
                               name=key,
                               description=data_desc)

# getting nba dataset
nba_df = nba_dataset.to_pandas_dataframe()
print(nba_df.head())

experiment_name = 'hd_nba_3pt'
experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
# creating cpu cluster unless it's already created
cluster_name = "proj-compute-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Cluster already created.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='./conda_dependencies.yml')

# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling(
    {
        '--C': choice(0.01, 0.1, 0.5, 1),
        '--max_iter': choice(20, 40, 80, 120, 160, 200)
    }
)

#TODO: Create your estimator and hyperdrive config
estimator = ScriptRunConfig(source_directory="./",
                            script='./train.py',
                            compute_target=cpu_cluster,
                            environment=sklearn_env)

hyperdrive_run_config = HyperDriveConfig(run_config=estimator,
                                         hyperparameter_sampling=param_sampling,
                                         policy=early_termination_policy,
                                         primary_metric_name='Accuracy',
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                         max_total_runs=20,
                                         max_concurrent_runs=4)

In [ ]:
#TODO: Submit your experiment
# run hyperdrive experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
# get run details
RunDetails(hyperdrive_run).show()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()
print('Best Run Id: ', best_run.id)
print('\n')
print('Best Run Metrics:', best_run_metrics)

In [ ]:
#TODO: Save the best model
joblib.dump(value=best_run.id, filename='outputs/hd-best-sdk.joblib')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
# register the model
model_name = 'nba_3pt_hyperdrive_lr'
model_description = 'Logistic Regression Model for NBA 3PT Percentage Win Analysis'
registered_model = hyperdrive_run.register_model(model_name=model_name, description=model_description)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

